In [ ]:
#Sentimental analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd https://drive.google.com/file/d/19vPHVZITAYxa6d-IR_KWqc11i3qmyMcZ/view?usp=drive_link
!ls

[Errno 2] No such file or directory: 'https://drive.google.com/file/d/19vPHVZITAYxa6d-IR_KWqc11i3qmyMcZ/view?usp=drive_link'
/content
amazon_alexa.tsv  drive  sample_data


In [ ]:
import pandas as pd

data = pd.read_csv('amazon_alexa.tsv', sep='\t')
data.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [ ]:
mydata = data[['verified_reviews','feedback']]
mydata.columns = ['review','label']

mydata.head()

,review,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
mydata.value_counts('label')

,count
label,
1,2893
0,257


In [ ]:
#count the occurence of each label
label_counts = mydata['label'].value_counts()

#get the no. of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

#drop rows from the majority class randomly
if rows_to_drop > 0:
  data_majority = mydata[mydata["label"] == 1]
  data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
  data_balanced = mydata.copy()

#check the new class balance
print(data_balanced['label'].value_counts())

label
1    257
0    257
Name: count, dtype: int64


In [ ]:
#Data Processing

In [ ]:
import re

def clean_text(text):
  text = str(text)
  text = re.sub(r"[^\w\s]", " ", text)

  text = re.sub(r"\b[a-zA-z]\b", " ", text)

  text = text.lower()

  text = re.sub(r"\s+", " ", text)

  text = text.strip()

  return text

In [ ]:
import pandas as pd
reviews = data_balanced['review'].tolist()

cleaned_reviews = [clean_text(review) for review in reviews ]

data_balanced['clean_reviews'] = cleaned_reviews
data_balanced.head()

,review,label,clean_reviews
9,Love it! I’ve listened to songs I haven’t hear...,1,love it ve listened to songs haven heard since...
11,I love it! Learning knew things with it eveyda...,1,love it learning knew things with it eveyday s...
40,My husband likes being able to use it to liste...,1,my husband likes being able to use it to liste...
46,"It's like Siri, in fact, Siri answers more acc...",0,it like siri in fact siri answers more accurat...
62,Did t really know what I could use this device...,1,did really know what could use this device for...


In [ ]:
#data spliting


In [ ]:
import pandas as pd
total_rows = len(data_balanced)
test_size = int(total_rows * 0.95)
test_set = data_balanced.sample(test_size)
train_set = data_balanced.drop(test_set.index)


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('.', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
response = model.generate_content("what is the meaning of life?")
to_markdown(response.text)

CPU times: user 105 ms, sys: 21.3 ms, total: 127 ms
Wall time: 7.11 s


> The meaning of life is a deeply personal and subjective question, and there is no one universally accepted answer * However, some common perspectives on the meaning of life include:
> 
> * **To find happiness and fulfillment:** Many people believe that the meaning of life is to find things that bring them joy and satisfaction, whether that's spending time with loved ones, pursuing their passions, or making a difference in the world *
> * **To contribute to something greater than oneself:** Others believe that the meaning of life is to use their talents and abilities to make the world a better place, whether that's through volunteering, activism, or simply being a kind and compassionate person *
> * **To learn and grow:** Life can be seen as a journey of learning and growth, and some people believe that the meaning of life is to gain knowledge, experiences, and wisdom that help them become better people *
> * **To connect with others:** Relationships are an important part of life, and some people believe that the meaning of life is to build strong and meaningful connections with others *
> * **To live in the present moment:** Life is constantly changing, and some people believe that the meaning of life is to appreciate the present moment and savor the simple things that make life worth living *
> 
> Ultimately, the meaning of life is different for each individual * It's something that can be explored and discovered through personal reflection, experiences, and interactions with others *

In [ ]:
test_set_sample = test_set.sample(20)
test_set_sample['pred_label'] = ''
test_set_sample

,review,label,clean_reviews,pred_label
1084,Easy setup Love it!,1,easy setup love it,
566,The second one which was a refurbished model d...,0,the second one which was refurbished model did...,
1059,Honestly I like it but at the same time I don’...,0,honestly like it but at the same time don coul...,
1200,Meh,0,meh,
2929,"We really love our Echo Dot, we can get questi...",1,we really love our echo dot we can get questio...,
1879,I’ve been waiting for Prime Day to purchase an...,1,ve been waiting for prime day to purchase an e...,
2305,Had to return it as Hulu does not work on devi...,0,had to return it as hulu does not work on devi...,
664,Great for mom its inside an owlhead in her jac...,1,great for mom its inside an owlhead in her jac...,
2616,I love it! I use it for everything. It’s a par...,1,love it use it for everything it part of my da...,
2215,My Father liked the added programing in which ...,1,my father liked the added programing in which ...,


In [ ]:
json_data = test_set_sample[['clean_reviews','pred_label']].to_json(orient='records', lines= True)
print(json_data)

{"clean_reviews":"easy setup love it","pred_label":""}
{"clean_reviews":"the second one which was refurbished model did work at least not in my home","pred_label":""}
{"clean_reviews":"honestly like it but at the same time don could have bought smaller one that did the exact same thing for less then half bought it for the video chat and have never really used it","pred_label":""}
{"clean_reviews":"meh","pred_label":""}
{"clean_reviews":"we really love our echo dot we can get questions answered right away and play any music we can think of","pred_label":""}
{"clean_reviews":"ve been waiting for prime day to purchase an echo plus and now enjoying learning what alexa can do with new sonos one and an echo dot have the makings of really nice and well integrated smart home love it","pred_label":""}
{"clean_reviews":"had to return it as hulu does not work on device it kept coming up with errors and hulu app not being able to open at times it was hit and miss and at times hulu will stop workin

In [ ]:
prompt = f"""
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.


```
{json_data}
```
"""
print(prompt)


You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.


```
{"clean_reviews":"easy setup love it","pred_label":""}
{"clean_reviews":"the second one which was refurbished model did work at least not in my home","pred_label":""}
{"clean_reviews":"honestly like it but at the same time don could have bought smaller one that did the exact same thing for less then half bought it for the video chat and have never really used it","pred_label":""}
{"clean_reviews":"meh","pred_label":""}
{"clean_reviews":"we really love our echo dot we can get questions answered right away a

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
{"clean_reviews":"easy setup love it","pred_label":1}
{"clean_reviews":"the second one which was refurbished model did work at least not in my home","pred_label":0}
{"clean_reviews":"honestly like it but at the same time don could have bought smaller one that did the exact same thing for less then half bought it for the video chat and have never really used it","pred_label":0}
{"clean_reviews":"meh","pred_label":0}
{"clean_reviews":"we really love our echo dot we can get questions answered right away and play any music we can think of","pred_label":1}
{"clean_reviews":"ve been waiting for prime day to purchase an echo plus and now enjoying learning what alexa can do with new sonos one and an echo dot have the makings of really nice and well integrated smart home love it","pred_label":1}
{"clean_reviews":"had to return it as hulu does not work on device it kept coming up with errors and hulu app not being able to open at times it was hit and miss and at times hulu will stop working 

In [ ]:
import json
import pandas as pd

# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Assuming the response contains multiple JSON objects, split them by newline
json_objects = json_data.splitlines()

# Parse each JSON object and append to a list
data = []
for obj in json_objects:
    try:
        # Try parsing the current line as JSON
        parsed_obj = json.loads(obj)
        data.append(parsed_obj)  # Add the parsed object to the list
    except json.JSONDecodeError as e:
        # Handle any JSONDecodeErrors gracefully, you may log or print error messages
        print(f"Error parsing JSON object: {obj}\nError: {e}")

# Create a DataFrame from the list of parsed JSON objects
df_sample = pd.DataFrame(data)

df_sample

Error parsing JSON object: 
Error: Expecting value: line 1 column 1 (char 0)


,clean_reviews,pred_label
0,easy setup love it,1
1,the second one which was refurbished model did...,0
2,honestly like it but at the same time don coul...,0
3,meh,0
4,we really love our echo dot we can get questio...,1
5,ve been waiting for prime day to purchase an e...,1
6,had to return it as hulu does not work on devi...,0
7,great for mom its inside an owlhead in her jac...,1
8,love it use it for everything it part of my da...,1
9,my father liked the added programing in which ...,1


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_sample['pred_label'] = df_sample['pred_label'].values
test_set_sample

,review,label,clean_reviews,pred_label
1084,Easy setup Love it!,1,easy setup love it,1
566,The second one which was a refurbished model d...,0,the second one which was refurbished model did...,0
1059,Honestly I like it but at the same time I don’...,0,honestly like it but at the same time don coul...,0
1200,Meh,0,meh,0
2929,"We really love our Echo Dot, we can get questi...",1,we really love our echo dot we can get questio...,1
1879,I’ve been waiting for Prime Day to purchase an...,1,ve been waiting for prime day to purchase an e...,1
2305,Had to return it as Hulu does not work on devi...,0,had to return it as hulu does not work on devi...,0
664,Great for mom its inside an owlhead in her jac...,1,great for mom its inside an owlhead in her jac...,1
2616,I love it! I use it for everything. It’s a par...,1,love it use it for everything it part of my da...,1
2215,My Father liked the added programing in which ...,1,my father liked the added programing in which ...,1


In [ ]:
# Plotting confusion matrix on the predictions

from sklearn.metrics import confusion_matrix

y_true = test_set_sample["label"]
y_pred = test_set_sample["pred_label"]

confusion_matrix(y_true, y_pred)

array([[9, 0],
       [2, 9]])